<a href="https://colab.research.google.com/github/nojha95/Whatsapp_groupchat_analysis/blob/main/whatsapp_chat_analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import regex

In [2]:
import matplotlib.pyplot as plt
import plotly.express as px


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
file = '/gdrive/MyDrive/WhatsApp Chat with Trip trip trip_1.txt' 

In [5]:
 x = open(file,'r', encoding = 'utf-8')

In [6]:
y = x.read()

In [7]:
chat = y.splitlines()
print(len(chat))


929


Merge messages that belong together

For every line, see if it matches the expression which is starting with the format "number(s)+slash" eg "12/"
If it does, it is a new line of conversion as they begin with dates, add it to msgs container
Else, it is a continuation of the previous line, add it to the previous line and append to msgs, then pop previous line.

In [ ]:
clean_chat = chat[3:]
print(clean_chat)

In [9]:
msgs = []
pos =0
pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
for line in clean_chat:
  if re.findall("\A\d+[/]", line):
    msgs.append(line)
    pos += 1
  else:
    merge = msgs[pos-1] + ". " + line
    msgs.append(merge)
    msgs.pop(pos-1)

print(len(msgs))

816


Extract date and time component

In [ ]:
time = [msgs[i].split(',')[1].split('-')[0] for i in range(len(msgs))]
time = [s.strip(' ') for s in time]
print(time)
len(time)

In [ ]:
date = [msgs[i].split(',')[0] for i in range(len(msgs))]
print(date)
len(date)

In [ ]:
name = [msgs[i].split('-')[1].split(':')[0] for i in range(len(msgs))]
name = [n.strip(' ') for n in name]
print(name)
len(name)

In [ ]:
content = []
for i in range(len(msgs)):
  content.append(msgs[i].split(":")[2])

content = [c.strip(' ') for c in content]
print(content)
len(content)

Create pandas data frame

In [14]:
df = pd.DataFrame(list(zip(date,time,name,content)),columns= ["Date","Time","Name","Content"])

In [ ]:
df

In [ ]:
df = df.dropna()
df

In [17]:
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

Remove media messages


In [18]:
df_media = df[df['Content'] == "<Media omitted>"]
df_media.shape

(42, 5)

In [96]:
df_messages = df.drop(df_media.index)
df_messages.shape


(774, 5)

Add word cound and length to messages dataframe



In [ ]:
df_messages["Length"] = df_messages['Content'].apply(lambda x : (len(x.replace(" ",""))))
df_messages

In [ ]:
df_messages["Words"] = df_messages["Content"].apply(lambda x : len(x.split(" ")))
df_messages

In [22]:
type(df_messages)

pandas.core.frame.DataFrame

Get Analytics - Member wise


In [23]:
members = df_messages['Name'].unique()
members

array(['Vandy', 'Baruni', 'N O', 'Aditi', 'Kartik M', 'Harshit M'],
      dtype=object)

In [24]:
avg_message = []
avg_words = []
ind_text_msg = []
for member in members:
  avg_message.append(round(df_messages[df_messages["Name"] == member]["Length"].mean()))
  avg_words.append(round((df_messages[df_messages["Name"] == member]["Words"]).mean()))
  ind_text_msg.append(df_messages[df_messages["Name"] == member]["Name"].count())
  

print(avg_words)
ind_text_msg
print(avg_message)

[6, 5, 7, 5, 7, 3]
[24, 19, 30, 21, 30, 16]


In [25]:
media_sent = []
for member in members:
  media_sent.append(df_media[df_media["Name"] == member]["Name"].count())

media_sent


[14, 12, 7, 8, 1, 0]

In [91]:
plt_1 = px.bar(x = members,y = ind_text_msg,labels= {"x":"Name","y":"Message Count"},width = 500, color = members,title = "Total Messages sent")
plt_1.update_layout(showlegend=False)
plt_1.show()

In [92]:
plt_2 = px.bar(x = members,y = media_sent,labels= {"x":"Name","y":"Media Count"},width = 500, color = members,title = "Total Media sent")
plt_2.update_layout(showlegend=False)
plt_2.show()

In [93]:

plt_2 = px.bar(x = members,y = avg_message,labels= {"x":"Name","y":"Average char count"},width = 500, color = members,title = "Average message length")
plt_2.update_layout(showlegend=False)
plt_2.show()

In [94]:
plt = px.bar(x = members,y = avg_words,labels= {"x":"Name","y":"Average words"},width = 500, color = members,title = "Average words in message")
plt.update_layout(showlegend=False)
plt.show()

In [ ]:
max_length = df_messages["Length"].max()
longest_message = df_messages[df_messages["Length"] == max_length]
longest_message

In [73]:
date_df = df_messages.groupby("Date").count()
date_df.reset_index(inplace=True)
date_df["Date_d"] = pd.to_datetime(date_df['Date'],format='%d/%m/%y')
date_df = date_df.sort_values(by='Date_d')

In [74]:
fig = px.line(date_df, x="Date_d", y="Name",  title='Number of Messages as time moves on.',labels={ "Date_d" : "Date","Name": "Message Count"})
fig.update_xaxes(nticks=20)
fig.show()

In [89]:
df_name = date_df.sort_values(by = "Content",ascending=False)
plt_3 = px.bar(x = df_name["Date"].head(10),y = df_name["Content"].head(10),labels = {"x":"Date","y":"Message Count"},title = "Busiest Days")
plt_3.show()